In [1]:
import numpy as np
from os.path import normpath as fn
import autograd 

np.random.seed(0)

In [3]:
# Load data
data = np.load(fn('data/mnist_26k.npz'))
train_im = np.float32(data['im_train'])/255.-0.5
train_im = np.reshape(train_im,[-1,28,28,1])
train_lb = data['lbl_train']

val_im = np.float32(data['im_val'])/255.-0.5
val_im = np.reshape(val_im,[-1,28,28,1])
val_lb = data['lbl_val']

In [4]:
# Init Weights
def xavier(shape):
    sq = np.sqrt(3.0/np.prod(shape[:-1]))
    return np.random.uniform(-sq,sq,shape)

In [6]:
# Inputs and parameters
inp = autograd.Value()
lab = autograd.Value()

K1 = autograd.Param()
B1 = autograd.Param()

K2 = autograd.Param()
B2 = autograd.Param()

W3 = autograd.Param()
B3 = autograd.Param()


# Model
y = autograd.conv2(inp,K1,1)
y = autograd.down2(y)
y = autograd.add(y,B1)
y = autograd.RELU(y)

y = autograd.conv2(y,K2,1)
y = autograd.down2(y)
y = autograd.add(y,B2)
y = autograd.RELU(y)


y = autograd.flatten(y)

y = autograd.matmul(y,W3)
y = autograd.add(y,B3) # This is our final prediction


# Cross Entropy of Soft-max
loss = autograd.smaxloss(y,lab)
loss = autograd.mean(loss)

# Accuracy
acc = autograd.accuracy(y,lab)
acc = autograd.mean(acc)

In [7]:
C1 = 8
C2 = 16

K1.set(xavier((4,4,1,C1)))
B1.set(np.zeros((C1)))

K2.set(xavier((2,2,C1,C2)))
B2.set(np.zeros((C2)))

W3.set(xavier((C2*36,10)))
B3.set(np.zeros((10)))

In [8]:
# Training loop
BSZ=50
lr=0.001

NUM_EPOCH=10
DISPITER=50
batches = range(0,len(train_lb)-BSZ+1,BSZ)

## Implement Momentum and uncomment following line
autograd.init_momentum()


niter=0; avg_loss = 0.; avg_acc = 0.
for ep in range(NUM_EPOCH+1):

    # As we train, let's keep track of val accuracy
    vacc = 0.; vloss = 0.; viter = 0
    for b in range(0,len(val_lb)-BSZ+1,BSZ):
        print(val_im[b:b+BSZ,...].shape)
        inp.set(val_im[b:b+BSZ,...]); lab.set(val_lb[b:b+BSZ])
        autograd.Forward()
        viter = viter + 1;vacc = vacc + acc.top;vloss = vloss + loss.top
    vloss = vloss / viter; vacc = vacc / viter * 100
    print("%09d: #### %d Epochs: Val Loss = %.3e, Accuracy = %.2f%%" % (niter,ep,vloss,vacc))
    if ep == NUM_EPOCH:
        break

    # Shuffle Training Set
    idx = np.random.permutation(len(train_lb))

    # Train one epoch
    for b in batches:
        # Load a batch
        inp.set(train_im[idx[b:b+BSZ],...])
        lab.set(train_lb[idx[b:b+BSZ]])

        edf.Forward()
        avg_loss = avg_loss + loss.top; avg_acc = avg_acc + acc.top;
        niter = niter + 1
        if niter % DISPITER == 0:
            avg_loss = avg_loss / DISPITER; avg_acc = avg_acc / DISPITER * 100
            print("%09d: Training Loss = %.3e, Accuracy = %.2f%%" % (niter,avg_loss,avg_acc))
            avg_loss = 0.; avg_acc = 0.;

        autograd.Backward(loss)
        #edf.SGD(lr)
        autograd.momentum(lr,0.9)

AttributeError: 'Param' object has no attribute 'top'